In [142]:
import pandas as pd
import os
from zipfile import ZipFile 
import re
import collections
import numpy as np
import xmltodict
import gzip
import pubchempy as pcp

In [16]:
ENSP_geneid_path = os.path.join('../data/raw', 'ENSP_geneid.csv')
ENSP_geneid = pd.read_csv(ENSP_geneid_path)

In [17]:
gene_mapping = {ensp:geneid for (ensp, geneid) in zip(ENSP_geneid['initial_alias'], (ENSP_geneid['name']))}

In [56]:
db_vocab = pd.read_csv('../data/raw/drugbank_vocabulary.csv')
db_vocab['Common name'] = db_vocab['Common name'].str.lower()
db_vocab['Synonyms'] = db_vocab['Synonyms'].str.lower()
db_vocab

,DrugBank ID,Accession Numbers,Common name,CAS,UNII,Synonyms,Standard InChI Key
0,DB00001,BIOD00024 | BTD00024,lepirudin,138068-37-8,Y43GF64R34,hirudin variant-1 | lepirudin recombinant,NaN
1,DB00002,BIOD00071 | BTD00071,cetuximab,205923-56-4,PQX0D8J21J,cetuximab | cétuximab | cetuximabum,NaN
2,DB00003,BIOD00001 | BTD00001,dornase alfa,143831-71-4,953A26OA1Y,deoxyribonuclease (human clone 18-1 protein mo...,NaN
3,DB00004,BIOD00084 | BTD00084,denileukin diftitox,173146-27-5,25E79B5CTM,denileukin | interleukin-2/diptheria toxin fus...,NaN
4,DB00005,BIOD00052 | BTD00052,etanercept,185243-69-0,OP401G7OJC,etanercept | etanercept-szzs | etanercept-ykro...,NaN
...,...,...,...,...,...,...,...
13558,DB15672,NaN,rilematovir,1383450-81-4,NQ99E8OH3P,rilematovir,GTQTUABHRCWVLL-UHFFFAOYSA-N
13559,DB15673,NaN,lenacapavir,2189684-44-2,A9A0O6FB4H,lenacapavir,BRYXUCLEHAUSDY-WEWMWRJBSA-N
13560,DB15674,NaN,sisunatovir,1903763-82-5,KE63TTO7WK,sisunatovir,JOPCJJSYRPUEDS-UHFFFAOYSA-N
13561,DB15675,NaN,baloxavir,1985605-59-1,4G86Y4JT3F,baloxavir,FIDLLEYNNRGVFR-CTNGQTDRSA-N


In [186]:
sider_names = pd.read_csv('../data/raw/sider_drug_names.tsv', sep='\t', header=None)
sider_names[0] = sider_names[0].str.split('CID1',expand=True)[1]
sider_names_dict = {drug_id:drug_name for (drug_id, drug_name) in zip(sider_names[0], sider_names[1])}

### get possible genes, disease drugs from other file

In [5]:
all_genes = pd.read_csv('../data/clean/genes.nodes',header=None)

In [6]:
all_genes.head()

,0
0,A1BG
1,A1CF
2,AACS
3,AADAC
4,AADACL2


In [7]:
all_disease = pd.read_csv('../data/clean/diseases.nodes',header=None)
all_disease.head()

,0
0,C0000737
1,C0000810
2,C0000880
3,C0001126
4,C0001144


In [8]:
all_drugs = pd.read_csv('../data/clean/drugs.nodes',header=None)
all_drugs

,0
0,1
1,137
2,174
3,176
4,204
...,...
2518,139031013
2519,139266768
2520,139595263
2521,145712321


### Disgenet

In [9]:
disgenet_path = os.path.join('../data/raw', 'curated_gene_disease_associations.tsv.gz')
with gzip.open(disgenet_path) as file:
    disgenet = pd.read_csv(file, sep='\t', engine='python')

In [10]:
disgenet = disgenet[['geneSymbol', 'diseaseId', 'diseaseSemanticType']]

In [11]:
# filter genes
disgenet = disgenet[(disgenet['geneSymbol'].isin(all_genes[0]))]

# filter disease
disgenet = disgenet[disgenet['diseaseId'].isin(all_disease[0])]

In [12]:
disgenet

,geneSymbol,diseaseId,diseaseSemanticType
1,A1BG,C0036341,Mental or Behavioral Dysfunction
28,NAT1,C0001973,Mental or Behavioral Dysfunction
30,NAT1,C0005684,Neoplastic Process
37,NAT1,C0033578,Neoplastic Process
41,NAT1,C0376358,Neoplastic Process
...,...,...,...
81564,OCLN,C0006142,Neoplastic Process
81567,OCLN,C0007820,Disease or Syndrome
81568,OCLN,C0019156,Disease or Syndrome
81576,OCLN,C0678222,Neoplastic Process


In [ ]:
disgenet[['geneSymbol', 'diseaseId']].to_csv('../data/clean/gene-disease-association.edges',header=None,index=False)

After filtering, only 57526 overlap with STITCH and drugcentral

### Use Side effect data from Decagon

In [314]:
decagon_path = os.path.join('../data/raw', 'bio-decagon-combo.tar.gz')
with gzip.open(decagon_path) as file:
    decagon = pd.read_csv(file)
decagon.columns = ['drug1','drug2','se_id', 'se']

In [315]:
decagon = decagon.dropna()
duplicate = decagon[['drug1', 'drug2']]
decagon = duplicate.drop_duplicates()

In [316]:
decagon['drug1'] = decagon['drug1'].str.split('CID0',expand=True)[1]
decagon['drug2'] = decagon['drug2'].str.split('CID0',expand=True)[1]

/home/haoxin/anaconda3/envs/pytorch-geometric/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/haoxin/anaconda3/envs/pytorch-geometric/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [298]:
# decagon['drug1'] = decagon['drug1'].map(sider_names_dict)
# decagon['drug2'] = decagon['drug2'].map(sider_names_dict)

In [317]:
decagon['drug1'] = decagon['drug1'].map(lambda x: int(x))
decagon['drug2'] = decagon['drug2'].map(lambda x: int(x))

/home/haoxin/anaconda3/envs/pytorch-geometric/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/haoxin/anaconda3/envs/pytorch-geometric/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [318]:
url = 'https://raw.githubusercontent.com/dhimmel/drugbank/3e87872db5fca5ac427ce27464ab945c0ceb4ec6/data/mapping/pubchem.tsv'
drugbank_map_df = pd.read_table(url)

In [319]:
drugbank_vocab_dict = {pubid:drugbank for (pubid,drugbank) in 
                       zip(drugbank_map_df['pubchem_id'], drugbank_map_df['drugbank_id'])}

map to Drugbank id

In [320]:
decagon['drug1'] = decagon['drug1'].map(drugbank_vocab_dict)
decagon['drug2'] = decagon['drug2'].map(drugbank_vocab_dict)
# decagon_final = decagon.dropna()

/home/haoxin/anaconda3/envs/pytorch-geometric/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/haoxin/anaconda3/envs/pytorch-geometric/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [324]:
decagon = decagon.dropna()

In [331]:
decagon_final = decagon.drop_duplicates()

In [334]:
decagon_final.to_csv('../data/clean/drug-drug-side_effects.edges',header=None, index=False)

### StringDB

In [18]:
stringdb_path = os.path.join('../data/raw', '9606.protein.actions.v11.0.txt.gz')
with gzip.open(stringdb_path) as file:
    stringdb_actions = pd.read_csv(file, sep='\s', engine='python')

In [19]:
stringdb_actions

,item_id_a,item_id_b,mode,action,is_directional,a_is_acting,score
0,9606.ENSP00000000233,9606.ENSP00000216366,binding,NaN,f,f,165
1,9606.ENSP00000000233,9606.ENSP00000216366,reaction,NaN,f,f,165
2,9606.ENSP00000000233,9606.ENSP00000216366,reaction,NaN,t,f,165
3,9606.ENSP00000000233,9606.ENSP00000216366,reaction,NaN,t,t,165
4,9606.ENSP00000000233,9606.ENSP00000222547,binding,NaN,f,f,913
...,...,...,...,...,...,...,...
3470901,9606.ENSP00000485678,9606.ENSP00000409581,inhibition,inhibition,f,f,600
3470902,9606.ENSP00000485678,9606.ENSP00000409581,ptmod,NaN,f,f,600
3470903,9606.ENSP00000485678,9606.ENSP00000438346,activation,activation,t,f,900
3470904,9606.ENSP00000485678,9606.ENSP00000481878,activation,activation,f,f,600


In [20]:
pd.unique(stringdb_actions['mode'])

array(['binding', 'reaction', 'catalysis', 'activation', 'inhibition',
       'ptmod', 'expression'], dtype=object)

In [21]:
stringdb_actions = stringdb_actions[['item_id_a', 'item_id_b', 'mode']]
stringdb_actions.columns = ['gene1','gene2','mode']

In [22]:
stringdb_actions['gene1'] = stringdb_actions['gene1'].str.split('.',expand=True)[1]
stringdb_actions['gene2'] = stringdb_actions['gene2'].str.split('.',expand=True)[1]

/home/haoxin/anaconda3/envs/pytorch-geometric/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/haoxin/anaconda3/envs/pytorch-geometric/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [23]:
stringdb_actions['gene1'] = stringdb_actions['gene1'].map(gene_mapping)
stringdb_actions['gene2'] = stringdb_actions['gene2'].map(gene_mapping)

/home/haoxin/anaconda3/envs/pytorch-geometric/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/haoxin/anaconda3/envs/pytorch-geometric/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [24]:
# filter genes 1
stringdb_actions = stringdb_actions[(stringdb_actions['gene1'].isin(all_genes[0]))]

# filter disease
stringdb_actions = stringdb_actions[(stringdb_actions['gene2'].isin(all_genes[0]))]

In [25]:
stringdb_actions = stringdb_actions.drop_duplicates()

In [26]:
stringdb_actions.to_csv('../data/clean/gene-gene-association.edges',header=None, index=False)